## Cleaning Salesforce Input
This project cleans a spreadsheet for a client to upload a long list of vendors into Salesforce.

In [1]:
import pandas as pd
import numpy as np
import re

The spreadsheet has separate worksheets for every state. We want to load each worksheet into its own dataframe for easy concatenation. Start by initilizing an empty dictionary.

In [2]:
# Load a csv with state abbreviations
states_df = pd.read_csv('C:\\Users\\avery\\OneDrive\\TBTN\\state_abbreviations.csv')

# Select just the abbreviations 
states_series = states_df['code']

# Initilize a dictionary from the series
raw_df_dict = dict.fromkeys(states_series, 0)

print(raw_df_dict)

{'AL': 0, 'AK': 0, 'AZ': 0, 'AR': 0, 'CA': 0, 'CO': 0, 'CT': 0, 'DE': 0, 'FL': 0, 'GA': 0, 'HI': 0, 'ID': 0, 'IL': 0, 'IN': 0, 'IA': 0, 'KS': 0, 'KY': 0, 'LA': 0, 'ME': 0, 'MD': 0, 'MA': 0, 'MI': 0, 'MN': 0, 'MS': 0, 'MO': 0, 'MT': 0, 'NE': 0, 'NV': 0, 'NH': 0, 'NJ': 0, 'NM': 0, 'NY': 0, 'NC': 0, 'ND': 0, 'OH': 0, 'OK': 0, 'OR': 0, 'PA': 0, 'RI': 0, 'SC': 0, 'SD': 0, 'TN': 0, 'TX': 0, 'UT': 0, 'VT': 0, 'VA': 0, 'WA': 0, 'WV': 0, 'WI': 0, 'WY': 0}


Define a function to load one worksheet given an input.

In [3]:
def load_states(state):
    
    # Specify file path
    file_path = 'C:\\Users\\avery\\OneDrive\\TBTN\\manual_studio_spreadsheet.xlsx'
    
    # Load Excel worksheet with state name
    temp_df = pd.read_excel(file_path, sheet_name=state)
    
    # Store in dictionary as value for state key
    raw_df_dict[state] = temp_df

Loop over the dictionary, loading each worksheet into the appropriate value.

In [4]:
for key in raw_df_dict:
    load_states(key)

print(raw_df_dict['OH'].columns)

Index(['STATUS', 'Studio Name', 'Address', 'Address Line 2', 'City', 'State',
       'Zip', 'Phone number', 'Unnamed: 8', 'Email', 'Website',
       'Owner First Name', 'Owner Last Name', 'Past Participant (Y/N)',
       'Years of Participation', 'WHO CONTACTED & EMAIL', 'DATE CONTACTED',
       'FOLLOW UP DATE', 'YES/MAYBE/NO', 'Signed Agreement?', '$ Raised',
       'NOTES', 'Salesforce', 'Salesforce Status'],
      dtype='object')


Concatenate all dataframes together.

In [5]:
combined_df = pd.concat(raw_df_dict, axis=0, ignore_index=True, copy=False)

print(combined_df.shape)

(3556, 61)


We know we have too many columns. Some we may need to adjust manually. Let's export a dataframe that contains the columns and the number of values in each.

In [6]:
export = pd.DataFrame(combined_df.count(), columns=['counts'])

export['fixed_status'] = False

export.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\columns_check.csv')

With our column counts and a view into the data with Excel, we can perform a series of cleaning steps.

### Drop unneeded columns

First, drop all columns we know we don't need: either they have no values, incoherent values, or the client has expressed they don't care about.

In [7]:
drop_list = ['Saleforce Status', 
            'Unnamed: 13', 
            'Unnamed: 14', 
            'Unnamed: 18', 
            'Unnamed: 19', 
            'Unnamed: 20',
            'Unnamed: 21',
            'Unnamed: 24',
            'Salesforce Notes',
            '$ Raised',
            'Signed Agreement?',
            'Unnamed: 0',
            'Follow-Up Date ',
            'Date Contacted',
            'YES/MAYBE/NO',
            'FOLLOW UP DATE',
            'DATE CONTACTED',
            'WHO CONTACTED & EMAIL',
            'Who ContACTED & EMAIL',
            'Status for Studio Outreach',
            '2nd Follow Up date']

counts_only_df = combined_df.drop(drop_list, axis=1)

print(counts_only_df.columns)

Index(['STATUS', 'Studio Name', 'Address', 'City', 'State', 'Zip',
       'Phone number', 'Email', 'Website', 'Owner First Name',
       'Owner Last Name', 'Past Participant Notes', 'Years of Participation',
       'NOTES', 'Salesforce', 'Salesforce Status', 'Address Line 2',
       'Past Participant (Y/N)', ' ', 'Unnamed: 8', 'NOTES.1',
       'Sent Free Webinar Email', 'Group/ Business Name ', 'City/ Village ',
       'Zipcode ', 'Phone Number ', 'Email for Website ',
       'Social Media Account', 'Temporarily closed Covet 19', 'Notes ',
       'Free Webinar Email', 'Salesforce ', 'Salesforce Status ',
       'Free Webinar email', 'Zip (they all have 0 in front)',
       'In Salesforce?', 'f', 'Free Webinar', 'STATUS - Amy's Marketing',
       'Marketing Email'],
      dtype='object')


We can see that there are several categories that have multiple columns. We should combine them into one column per category.

Let's define a function to do that.

In [8]:
def consolidate(df, source_columns, target_column):
    
    # Print the original column count
    origin_count = df[source_columns].count()
    print('Original: ' + str(origin_count))
    target_count = df[target_column].count()
    print('Target: ' + str(target_count))
    
    # Transform column list into a series so it can be used to fill na
    column_series = df[source_columns]
    
    # Fill na values from other lists, storing in new df
    result_df = df
    
    result_df[target_column] = df[target_column].fillna(column_series)
    
    # Print output column counts
    source_count = result_df[source_columns].count()
    result_count = result_df[target_column].count()
    
    # Print results
    print('Source: ' + str(source_count))
    print('Result: ' + str(result_count))
    
    # Return new df
    return result_df
    

consolidate(counts_only_df, 'In Salesforce?', 'Salesforce')

### Consolidate Repeated Columns

Call `consolidate()` on each category we need to combine. We should do this with a list or dictionary and a loop, but the function isn't working with that method, so let's just save troubleshooting time and do it manually.

In [9]:
sales_fix_1 = consolidate(counts_only_df, 'In Salesforce?', 'Salesforce')

Original: 2
Target: 158
Source: 2
Result: 160


In [10]:
sales_fix_2 = consolidate(sales_fix_1, 'Salesforce ', 'Salesforce')

Original: 7
Target: 160
Source: 7
Result: 167


In [11]:
status_fix_1 = consolidate(sales_fix_2, 'Salesforce Status ', 'Salesforce Status')

Original: 8
Target: 157
Source: 8
Result: 165


In [12]:
city_fix = consolidate(status_fix_1, 'City/ Village ', 'City')

Original: 71
Target: 3364
Source: 71
Result: 3435


In [13]:
zip_fix_1 = consolidate(city_fix, 'Zipcode ', 'Zip')

Original: 71
Target: 2743
Source: 71
Result: 2814


In [14]:
zip_fix_2 = consolidate(zip_fix_1, 'Zip (they all have 0 in front)', 'Zip')

Original: 41
Target: 2814
Source: 41
Result: 2855


Looks like this fix isn't working. Let's make a note to work this out manually and then move on.

In [15]:
notes_fix_1 = consolidate(zip_fix_2, 'Notes ', 'NOTES')

Original: 65
Target: 830
Source: 65
Result: 895


In [16]:
notes_fix_2 = consolidate(notes_fix_1, 'Past Participant Notes', 'NOTES')

Original: 5
Target: 895
Source: 5
Result: 900


In [17]:
notes_fix_3 = consolidate(notes_fix_2, 'f', 'NOTES')

Original: 67
Target: 900
Source: 67
Result: 935


In [18]:
notes_fix_4 = consolidate(notes_fix_3, 'Unnamed: 8', 'NOTES')

Original: 43
Target: 935
Source: 43
Result: 976


In [19]:
webinar_fix_1 = consolidate(notes_fix_4, 'Sent Free Webinar Email', 'Free Webinar Email')

Original: 10
Target: 20
Source: 10
Result: 30


In [20]:
webinar_fix_2 = consolidate(webinar_fix_1, 'Free Webinar', 'Free Webinar Email')

Original: 11
Target: 30
Source: 11
Result: 41


In [21]:
webinar_fix_3 = consolidate(webinar_fix_2, 'Free Webinar email', 'Free Webinar Email')

Original: 17
Target: 41
Source: 17
Result: 58


In [22]:
phone_fix_1 = consolidate(webinar_fix_3, ' ', 'Phone number')

Original: 42
Target: 3292
Source: 42
Result: 3334


In [23]:
phone_fix_2 = consolidate(phone_fix_1, 'Phone Number ', 'Phone number')

Original: 70
Target: 3334
Source: 70
Result: 3404


In [24]:
studio_fix_1 = consolidate(phone_fix_2, 'Group/ Business Name ', 'Studio Name')

Original: 71
Target: 3462
Source: 71
Result: 3533


In [25]:
notes_fix_5 = consolidate(studio_fix_1, 'NOTES.1', 'NOTES')

Original: 54
Target: 976
Source: 54
Result: 978


In [26]:
email_fix = consolidate(notes_fix_5, 'Email for Website ', 'Email') 

Original: 29
Target: 2843
Source: 29
Result: 2872


Now that we're finished moving values, let's drop the above columns from the dataframe.

In [27]:
drop_second_list = ['In Salesforce?',
                    'Salesforce ', 
                    'Salesforce Status ', 
                    'City/ Village ', 
                    'Zipcode ', 
                    'Notes ',
                    'Past Participant Notes',
                    'f',
                    'Unnamed: 8',
                    'Sent Free Webinar Email',
                    'Free Webinar',
                    'Free Webinar email',
                    'Phone Number ',
                    'Group/ Business Name ',
                    'NOTES.1',
                    'Email for Website ', 
                    'Zip (they all have 0 in front)',
                    "STATUS - Amy's Marketing", 
                    'Marketing Email', 
                    'Free Webinar Email',
                    ' '
                   ]

good_col_df = email_fix.drop(drop_second_list, axis=1)

print(notes_fix_5.shape)
print(good_col_df.shape)

(3556, 40)
(3556, 19)


In [28]:
good_col_df.columns

Index(['STATUS', 'Studio Name', 'Address', 'City', 'State', 'Zip',
       'Phone number', 'Email', 'Website', 'Owner First Name',
       'Owner Last Name', 'Years of Participation', 'NOTES', 'Salesforce',
       'Salesforce Status', 'Address Line 2', 'Past Participant (Y/N)',
       'Social Media Account', 'Temporarily closed Covet 19'],
      dtype='object')

### Combine Address Lines

Because Salesforce only accepts one address field, the client asked to have the two address fields combined. Let's do that now.

In [29]:
address_fix = good_col_df

# Concatenate strings
address_fix['Address'] = address_fix['Address'].astype(str) + ' ' + address_fix['Address Line 2'].astype(str)

# drop second column
address_fix.drop('Address Line 2', axis=1, inplace=True)

print(address_fix.columns)

Index(['STATUS', 'Studio Name', 'Address', 'City', 'State', 'Zip',
       'Phone number', 'Email', 'Website', 'Owner First Name',
       'Owner Last Name', 'Years of Participation', 'NOTES', 'Salesforce',
       'Salesforce Status', 'Past Participant (Y/N)', 'Social Media Account',
       'Temporarily closed Covet 19'],
      dtype='object')


### Strip trailing commas, spaces, and Nan from addresses

In [30]:
address_fix['Address'] = address_fix['Address'].str.rstrip('Nan')
address_fix['Address'] = address_fix['Address'].str.rstrip()
address_fix['Address'] = address_fix['Address'].str.rstrip(',')
address_fix['Address'] = address_fix['Address'].str.rstrip()

### Remove titles 
During the concatenation process some column titles got put in with the data. Remove them.

In [31]:
title_condition = address_fix['State'] == 'State'

initial_count = address_fix.loc[title_condition, :].count()

print(initial_count)

STATUS                         2
Studio Name                    2
Address                        2
City                           2
State                          2
Zip                            2
Phone number                   2
Email                          2
Website                        2
Owner First Name               2
Owner Last Name                2
Years of Participation         2
NOTES                          2
Salesforce                     0
Salesforce Status              0
Past Participant (Y/N)         2
Social Media Account           0
Temporarily closed Covet 19    0
dtype: int64


In [32]:
title_fix = address_fix.loc[~title_condition, :]

after_count = title_fix.loc[title_condition, :].count()

print(after_count)

STATUS                         0
Studio Name                    0
Address                        0
City                           0
State                          0
Zip                            0
Phone number                   0
Email                          0
Website                        0
Owner First Name               0
Owner Last Name                0
Years of Participation         0
NOTES                          0
Salesforce                     0
Salesforce Status              0
Past Participant (Y/N)         0
Social Media Account           0
Temporarily closed Covet 19    0
dtype: int64


### Drop Duplicates

In [33]:
title_fix.duplicated(['Studio Name', 'Address', 'Website']).value_counts()

False    3529
True       25
dtype: int64

In [34]:
unique_fix = title_fix.drop_duplicates(subset=['Studio Name', 'Address', 'Website'])

print(unique_fix.duplicated(['Studio Name', 'Address', 'Website']).value_counts())

False    3529
dtype: int64


In [35]:
unique_fix.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\merged_check.csv')

### Fix Phone Numbers

Took this cool function from sackoverflow. https://stackoverflow.com/a/62219947/14456176

In [36]:
def parsephone(strphone):
    '''
    Pull out just the digits. Then do some simple formating.
    '''
    phn = ""
    for n in strphone:
        if n in "0123456789":
            phn += n
    if len(phn) == 10:  # add a 1 in front
        phn = "1" + phn
    if len(phn) != 11 or len(phn) <=6:
        return phn  # no hope of formating
    # format with dashes
    phn = "(" + phn[1:4] + ") " + phn[4:7] + "-" + phn[7:]
    return phn

In [37]:
print(unique_fix.loc[:, 'Phone number'])

0       (334) 787-2110
1       (251) 639-9030
2       (205) 540-5842
3       (205) 202-5758
4       (256) 405-9226
             ...      
3551    (307) 752-2858
3552    (307) 358-9888
3553    (307) 413-0441
3554    (208) 317-0994
3555      307.680.7762
Name: Phone number, Length: 3529, dtype: object


In [38]:
phone_format = unique_fix
phone_format.loc[:, 'Phone number'] = phone_format.loc[:, 'Phone number'].astype('str')

phone_format.loc[:, 'Phone number'] = phone_format.loc[:, 'Phone number'].apply(lambda x: parsephone(x))

C:\Users\avery\anaconda3\lib\site-packages\pandas\core\indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [39]:
print(phone_format.loc[:, 'Phone number'])

0       (334) 787-2110
1       (251) 639-9030
2       (205) 540-5842
3       (205) 202-5758
4       (256) 405-9226
             ...      
3551    (307) 752-2858
3552    (307) 358-9888
3553    (307) 413-0441
3554    (208) 317-0994
3555    (307) 680-7762
Name: Phone number, Length: 3529, dtype: object


In [40]:
phone_format.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\phone_check.csv')

### Remove totally blank rows

In [41]:
print(phone_format.isna().sum())

no_blanks = phone_format.dropna(subset=['Studio Name'])

print(no_blanks.isna().sum())

STATUS                         2290
Studio Name                       3
Address                           0
City                            101
State                            29
Zip                             676
Phone number                      0
Email                           661
Website                         908
Owner First Name               2501
Owner Last Name                2870
Years of Participation         3464
NOTES                          2554
Salesforce                     3362
Salesforce Status              3364
Past Participant (Y/N)         3445
Social Media Account           3460
Temporarily closed Covet 19    3460
dtype: int64
STATUS                         2288
Studio Name                       0
Address                           0
City                            100
State                            28
Zip                             675
Phone number                      0
Email                           659
Website                         907
Owner First Nam

### Rename Columns

In [42]:
print(no_blanks.columns)

Index(['STATUS', 'Studio Name', 'Address', 'City', 'State', 'Zip',
       'Phone number', 'Email', 'Website', 'Owner First Name',
       'Owner Last Name', 'Years of Participation', 'NOTES', 'Salesforce',
       'Salesforce Status', 'Past Participant (Y/N)', 'Social Media Account',
       'Temporarily closed Covet 19'],
      dtype='object')


In [43]:
new_names_dict = {'STATUS': 'Status',
                  'Phone number': 'Phone', 
                  'NOTES': 'Notes', 
                  'Salesforce':'Salesforce Uploaded',
                  'Salesforce Status':'Salesforce Type',
                  'Temporarily closed Covet 19':'Temporary COVID Closure', 
                  'Past Participant (Y/N)': 'Past Participant'
                  }

renamed = no_blanks.rename(new_names_dict, axis=1)

print(renamed.columns)

Index(['Status', 'Studio Name', 'Address', 'City', 'State', 'Zip', 'Phone',
       'Email', 'Website', 'Owner First Name', 'Owner Last Name',
       'Years of Participation', 'Notes', 'Salesforce Uploaded',
       'Salesforce Type', 'Past Participant', 'Social Media Account',
       'Temporary COVID Closure'],
      dtype='object')


### Capitalize Vendor Info

In [44]:
capitalized = renamed

capitalized['Studio Name'] = renamed['Studio Name'].str.title()

capitalized['Address'] = renamed['Address'].str.title()

print(capitalized['Studio Name'].head(5))

0                            7Synergy
1    Above And Beyond Hot Yoga Center
2                      Abundance Yoga
3                            Aero Joe
4                Back In Balance Yoga
Name: Studio Name, dtype: object


### Restructure Vendor Names 
The client requests that vendor names that start with "the" be transformed to "(The)". We'll copy the trick from the `parsephone()` function above and reconfigure it to fit this case.

In [45]:
def brace_the(origin_str):
    """ Reformats a string if it contains "The"
    """
    if origin_str[0:3] == 'The':
        result_str = '(' + origin_str[0:3] + ')' + origin_str[3:]
    else:
        result_str = origin_str
    
    return result_str

In [46]:
braced_df = capitalized

braced_df.loc[:, 'Studio Name'] = braced_df.loc[:, 'Studio Name'].apply(lambda x: brace_the(x))

print(braced_df['Studio Name'].head(5))

0                            7Synergy
1    Above And Beyond Hot Yoga Center
2                      Abundance Yoga
3                            Aero Joe
4                Back In Balance Yoga
Name: Studio Name, dtype: object


In [47]:
braced_df.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\brace_check.csv')

### Enforce response consistency
The data was entered manually by many different people, and so we want to ensure we have uniform data in the final version. For example, we will want to turn Booleans from "yes", "not yet" or "YES!!!" into True or False.

These are the important columns to enforce:

* Status
* Years of Participation 
* Salesforce Uploaded
* Salesforce Type
* Past Participant
* Social Media Account
* Temporary COVID Closure

In [48]:
print(braced_df['Status'].value_counts())

Emailed                                          286
contacted                                         81
contacted & FP                                    43
Done                                              42
REPEAT                                            39
                                                ... 
d                                                  1
mailbox not set up                                 1
DOne                                               1
Email/Phone                                        1
sent email+phone number has been disconnected      1
Name: Status, Length: 292, dtype: int64


Okay, it looks like `Status` is nowhere close to a Boolean. Instead, it's more like a notes column. Let's leave this alone for now. When we're talking with the client on how to enforce data consistency going forward, let's see what they want this column to look like. We'll suggest a Boolean and leave notes for the `Notes` columns.

In [49]:
print(braced_df['Years of Participation'].value_counts())

0      54
-       6
1.0     5
Name: Years of Participation, dtype: int64


`Years of Participation` looks much simpler. Change '-' and nulls to 0.

In [50]:
# Create conditions to find all '-' and nulls
years_hyphen_condition = braced_df['Years of Participation'] == '-'
years_null_condition = braced_df['Years of Participation'].isna()

# Locate all '-' and null values and set to '0'
braced_df.loc[years_hyphen_condition, 'Years of Participation'] = 0
braced_df.loc[years_null_condition, 'Years of Participation'] = 0

# Change datatype to int
braced_df['Years of Participation'] = braced_df['Years of Participation'].astype('int')

# Check output
braced_df['Years of Participation'].value_counts()

0    3521
1       5
Name: Years of Participation, dtype: int64

Check the column `Salesforce Uploaded`.

In [51]:
print(braced_df['Salesforce Uploaded'].value_counts())

yes                    86
2020-10-25 00:00:00    16
2020-08-18 00:00:00    12
2020-07-31 00:00:00     7
yes                     4
2020-08-01 00:00:00     4
2020-09-16 00:00:00     3
2020-06-29 00:00:00     3
2020-08-23 00:00:00     3
2020-06-24 00:00:00     2
2020-07-03 00:00:00     2
2020-08-17 00:00:00     2
2020-07-26 00:00:00     2
YES                     2
2020-07-23 00:00:00     2
2020-09-15 00:00:00     1
2020-08-07 00:00:00     1
YES                     1
2020-07-01 00:00:00     1
2020-07-30 00:00:00     1
2020-09-14 00:00:00     1
2020-11-04 00:00:00     1
2020-07-21 00:00:00     1
2020-07-19 00:00:00     1
2020-09-18 00:00:00     1
2020-08-29 00:00:00     1
2020-06-26 00:00:00     1
2020-07-28 00:00:00     1
2020-07-08 00:00:00     1
2020-09-12 00:00:00     1
2020-08-12 00:00:00     1
2020-09-17 00:00:00     1
Name: Salesforce Uploaded, dtype: int64


The timestamp of upload isn't important there. Change all non-null values to 'Yes' and all nulls to 'No'. (Normally we'd set this to True or False, but our client will probably feel more comfortable with 'Yes' or 'No'.)

In [52]:
# Create conditions
salesforce_not_null = braced_df['Salesforce Uploaded'].notnull()
salesforce_null = braced_df['Salesforce Uploaded'].isna()

# Set values to 'Yes' or 'No'
braced_df.loc[salesforce_not_null, 'Salesforce Uploaded'] = 'Yes'
braced_df.loc[salesforce_null, 'Salesforce Uploaded'] = 'No'

# Check output
print(braced_df['Salesforce Uploaded'].value_counts())

No     3359
Yes     167
Name: Salesforce Uploaded, dtype: int64


Now check `Salesforce Type`.

In [53]:
print(braced_df['Salesforce Type'].value_counts())

account       87
Account       55
F Account     12
F Account      4
Account        3
ACCOUNT        3
account        1
Name: Salesforce Type, dtype: int64


In [54]:
print(braced_df['Salesforce Type'].isna().sum())

3361


It appears there are two real distinct categories, Account and F Account. There are many options for how to make this happen. Since there are several types of errors but only a few total, let's use a dictionary approach to replace the values.

In [55]:
account_dict = {'account': 'Account', 
                'Account ': 'Account', 
                'ACCOUNT': 'Account', 
                'account ': 'Account', 
                'F Account ': 'F Account'}

braced_df['Salesforce Type'].replace(account_dict, inplace=True)

print(braced_df['Salesforce Type'].value_counts())

Account      149
F Account     16
Name: Salesforce Type, dtype: int64


Check `Past Participant`:

In [56]:
print(braced_df['Past Participant'].value_counts())

N                                                                                                               63
Y                                                                                                                4
YES                                                                                                              3
Che                                                                                                              2
No                                                                                                               1
general manager                                                                                                  1
Yes                                                                                                              1
left message; Left message 8/26, owner Roseann picked up phone, is interested. Sent her email with info 8/26     1
Jennings                                                                        

This looks like it should be Boolean. It's also labeled `'Past Participant (Y/N)'` originally, so it's safe to assume they want it to be Boolean. Again, because our client will be more comfortable with it, let's convert all these values into 'Yes' or 'No'

In [57]:
par_fix = braced_df

# Create condition for 'No' values
no_participant_list = ['Almost then dropped', 'N']
no_participant_condition = (braced_df['Past Participant'].isin(no_participant_list)) | (braced_df['Past Participant'].isna())

# Locate nos and set to 'No'
par_fix.loc[no_participant_condition, 'Past Participant'] = 'No'

# Create condition for 'Yes'
yes_participant_condition = braced_df['Past Participant'] != 'No'

# Locate yes values and set to 'Yes'
par_fix.loc[yes_participant_condition, 'Past Participant'] = 'Yes'

# Check output
print(par_fix['Past Participant'].value_counts())

No     3507
Yes      19
Name: Past Participant, dtype: int64


Check `Social Media Account`:

In [58]:
print(par_fix['Social Media Account'].value_counts())

print(par_fix['Social Media Account'].isna().sum())

Facebook                       61
Facebook                        3
Unable to Find Social Media     2
FaceBook                        2
No Social Media Found           1
Name: Social Media Account, dtype: int64
3457


Let's convert all to either 'Facebook' or null.

In [59]:
social_fix = par_fix

social_map = {'Facebook ': 'Facebook', 
              'FaceBook ': 'Facebook', 
              'Unable to Find Social Media': np.nan, 
              'No Social Media Found ': np.nan}

social_fix.loc[:, 'Social Media Account'].replace(social_map, inplace=True)

print(social_fix['Social Media Account'].value_counts())

Facebook    66
Name: Social Media Account, dtype: int64


Finally, check `Temporary COVID Closure`:

In [60]:
print(social_fix['Temporary COVID Closure'].value_counts())

Open              41
Closed            20
Open               7
Online Classes     1
Name: Temporary COVID Closure, dtype: int64


This should be Boolean, but let's stick to the terms that the client used. We could set a default value, but let's not add useless data unnecessarily.

In [61]:
closure_fix = social_fix

closure_map = {'Open ': 'Open', 
              'Online Classes': 'Open'}

closure_fix.loc[:, 'Temporary COVID Closure'].replace(closure_map, inplace=True)

print(closure_fix['Temporary COVID Closure'].value_counts())

Open       49
Closed     20
Name: Temporary COVID Closure, dtype: int64


Export the dataframe with unified values to see if other steps should be taken.

In [62]:
closure_fix.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\enforced_check.csv')

### Reformat nulls in Address
Convert 'Nan' in `Address` to 'None'.

In [63]:
address_clean = closure_fix

address_clean.loc[:, 'Address'] = closure_fix.loc[:, 'Address'].replace({'Nan':'None'})

### Enforce correct states

In [64]:
correct_states = address_clean

correct_state_map = {'Iowa':'IA', 
                     'Alabama': 'AL', 
                     'Kansas': 'KS', 
                     'Kankas ': 'KS', 
                     'Kansas ': 'KS'}

correct_states['State'] = correct_states['State'].replace(correct_state_map)

correct_states['State'] = correct_states['State'].str.upper()

print(correct_states['State'].value_counts())

MA     493
WA     248
OR     196
CA     151
NV     105
FL     103
IL      78
NY      77
TX      75
AL      75
OH      69
MN      67
NY      66
NM      65
KY      62
CT      61
CO      61
MT      60
TN      59
MS      59
PA      55
IA      55
MD      53
NC      52
AZ      51
NH      50
MO      49
NE      49
SC      48
NJ      48
LA      47
UT      45
GA      45
RI      43
WI      42
ME      42
VT      42
AR      41
MI      38
IN      37
WY      36
WV      35
ID      35
AK      34
OK      32
ND      30
VA      27
HI      26
DE      24
SD      24
KS      23
MN       4
NV       2
UT       1
VY       1
WY       1
FL       1
Name: State, dtype: int64


### Switch incorrect email/website fields
When reviewing the data output, I noticed that some states had their website and email fields switched. Let's make sure the right values are in the right column.

In [65]:
# Initialize new df
switched_df = pd.DataFrame(correct_states, copy=True)

# List the states with wrong fields
states_to_switch = ['NC', 'NM', 'NE', 'MT']

# Create general switch condition
switch_condition = switched_df['State'].isin(states_to_switch) & (switched_df['Email'].str.contains('@', regex=False) == False)

# PA just has a couple to switch. 
# Let's swap all website values that contain '@'.
# Create PA condition
pa_switch_condition = (switched_df['State'] == 'PA') & (switched_df['Website'].str.contains('@', regex=False) == True)

# Initialize source columns to pull from
switched_df['source_email'] = switched_df['Email']
switched_df['source_website'] = switched_df['Website']

# For all incorrect states, set 'Email' to 'source_website' and 'Website' to 'Source Email'
switched_df.loc[switch_condition, 'Email'] = switched_df.loc[switch_condition, 'source_website']
switched_df.loc[switch_condition, 'Website'] = switched_df.loc[switch_condition, 'source_email']

# Repeat with PA
switched_df.loc[pa_switch_condition, 'Email'] = switched_df.loc[pa_switch_condition, 'source_website']
switched_df.loc[pa_switch_condition, 'Website'] = switched_df.loc[pa_switch_condition, 'source_email']

# Drop source columns
switch_complete_df = switched_df.drop(['source_email', 'source_website'], axis=1)

# Check output
print(switch_complete_df.loc[switch_complete_df['State'] == 'NC', ['Email', 'Website']])


                              Email  \
2377         caryflowyoga@gmail.com   
2378      INFO@PURAVIDASTUDIONC.COM   
2379        info@republicofyoga.com   
2380        jennie@wisemindyoga.com   
2381                            NaN   
2382                            NaN   
2383         info@zenyogacenter.com   
2384           info@CaryHotYoga.com   
2385           info@bluelotusnc.com   
2386        info@blissbody-yoga.com   
2387          info@yogagardennc.com   
2388      info@yourbodymechanic.com   
2389   hotyogartpcarymorr@gmail.com   
2390            winter@barre-up.com   
2391    frank@ignitefitnessapex.com   
2392                            NaN   
2393                            NaN   
2394                            NaN   
2395            110YOGANC@GMAIL.COM   
2396                            NaN   
2397    info@midtownyogastudios.com   
2398    smilingcircleyoga@gmail.com   
2399       brighttreeyoga@gmail.com   
2400          cyndi@zaktihealth.com   
2401                     

### Enforce emails only in 'Email' column

For the `Email` column, replace any value that has no '@' symbol with null.

In [66]:
# Initialize new df
email_enforce = switch_complete_df

# Create condition
not_an_email = email_enforce['Email'].str.contains('@', regex=False) == False

# Check that condition works
print(not_an_email.value_counts())
print(email_enforce['Email'].isna().sum())

# Set values that don't meet the condition to null
email_enforce.loc[not_an_email, 'Email'] = np.nan

# Check output
print(email_enforce['Email'].isna().sum())

False    3153
True      373
Name: Email, dtype: int64
709
1082


### Polish City and Email string format
Fix capitalization on `'City'` and `'Email'`:

In [67]:
# Initialize new df
capital_polish_df = email_enforce

# Fix City
capital_polish_df['City'] = capital_polish_df['City'].str.title()

# Strip trailing commas and spaces
capital_polish_df['City'] = capital_polish_df['City'].str.lstrip()
capital_polish_df['City'] = capital_polish_df['City'].str.rstrip(',')
capital_polish_df['City'] = capital_polish_df['City'].str.rstrip()
capital_polish_df['City'] = capital_polish_df['City'].str.rstrip(',')
capital_polish_df['City'] = capital_polish_df['City'].str.rstrip()

capital_polish_df['Email'] = capital_polish_df['Email'].str.lower()

### Create logical column order

In [68]:
capital_polish_df.columns

Index(['Status', 'Studio Name', 'Address', 'City', 'State', 'Zip', 'Phone',
       'Email', 'Website', 'Owner First Name', 'Owner Last Name',
       'Years of Participation', 'Notes', 'Salesforce Uploaded',
       'Salesforce Type', 'Past Participant', 'Social Media Account',
       'Temporary COVID Closure'],
      dtype='object')

In [69]:
export_df = capital_polish_df[['Studio Name', 
                               'Address', 
                               'City', 
                               'State', 
                               'Zip', 
                               'Phone',
                               'Email', 
                               'Website', 
                               'Owner First Name', 
                               'Owner Last Name', 
                               'Social Media Account', 
                               'Past Participant',
                               'Years of Participation', 
                               'Status', 
                               'Notes', 
                               'Temporary COVID Closure', 
                               'Salesforce Uploaded',
                               'Salesforce Type',
                              ]]



In [70]:
export_df.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\clean_master_export.csv')

## Done!
The dataset is now relatively tidy and ready for the client.

## Revisions
After reviewing the product, the client made some clarifications and requests. I also caught some more issues I want to fix before calling it complete.

First, drop `'Salesforce Type'`:

In [71]:
review_df = pd.DataFrame(export_df, copy=True)

review_df.drop('Salesforce Type', axis=1, inplace=True)

Rename `'Status'` to indicate that it holds past data, then add a new column `'Contact Attempts'` to represent the data we want to collect in the future.

In [75]:
status_fix = review_df.rename({'Status':'status_notes_2020'}, axis=1)

status_fix['Contact Attempts'] = np.nan

print(status_fix.columns)

Index(['Studio Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Email',
       'Website', 'Owner First Name', 'Owner Last Name',
       'Social Media Account', 'Past Participant', 'Years of Participation',
       'status_notes_2020', 'Notes', 'Temporary COVID Closure',
       'Salesforce Uploaded', 'Contact Attempts'],
      dtype='object')


Fix a few capitalizations. Make 'Ymca' and 'Llc' all caps.

In [99]:
caps_fix = status_fix

print(caps_fix['Studio Name'].str.contains('ymca', case=False).sum())

caps_condition = caps_fix['Studio Name'].str.contains('ymca', case=False)

caps_fix.loc[caps_condition, 'Studio Name'] = caps_fix['Studio Name'].str.replace('Ymca','YMCA', case=True)

print(caps_fix['Studio Name'].str.contains('Ymca', case=True).sum())

12
0


In [101]:
llc_fix = pd.DataFrame(caps_fix, copy=True)

print(llc_fix['Studio Name'].str.contains('llc', case=False).sum())

llc_condition = llc_fix['Studio Name'].str.contains('llc', case=False)

llc_fix.loc[llc_condition, 'Studio Name'] = llc_fix['Studio Name'].str.replace('Llc','LLC', case=True)

print(llc_fix['Studio Name'].str.contains('Llc', case=True).sum())

39
0


## Ready for final export.

In [102]:
llc_fix.to_csv('C:\\Users\\avery\\OneDrive\\TBTN\\clean_master_revised.csv')